# Extractive Text Summerization using Sentence Transformer and KMeans Clustering Algorithm

## Import required libraries

In [1]:
import nltk
import pandas as pd
from nltk.cluster import KMeansClusterer
import numpy as np
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-roberta-base')

## Sample Article

In [2]:
article='''
This week's launch of Aston Martin’s new Formula 1 car was one of the most hyped events of the pre-season so far, as fans were intrigued by how the new-look AMR21 would be painted. Unlike the car launches that came before it, Aston Martin left very little to the imagination, releasing detailed shots of the entire car. The first thing to note is that the team spent both of its development tokens on redesigning the chassis, in order that it could unlock aerodynamic performance from the central portion of the car. This is, in part, a legacy of the team’s approach for 2020, having assimilated the overall design package of the previous year’s championship winning Mercedes including a more conventional position for the side-impact protection spars (SIPS). The low-slung arrangement, as introduced by Ferrari in 2017, is now considered critical from an aerodynamic perspective, with the sidepod inlet positioned much like a periscope. This is typically above the fairing that surrounds the SIPS, which is used to inhibit the turbulence created by the front tyre and therefore also aids the transit of cool air that’s supplied to the radiators within the sidepods. This image of the car depicts how the bargeboards are used to filter the turbulence created by the front tyre and convert it into something more usable. Meanwhile, the airflow fed from the front of the car, including the cape, is forced around the underside of the sidepod whilst the fairing around the SIPS shields the airflow entering the sidepod inlet. This should result in a much cleaner flow arriving at the radiators, with the air having not been worked too hard by numerous surfaces en route. The inlet itself is extremely narrow with the team recovering some of that with the sculpting on the sides of the chassis. The narrowness of the inlet also draws your attention to the substantial fin that grows out of the sidepod’s shoulder and helps to divert airflow down over the revamped sidepod packaging behind. This is an area where the team has clearly focused its resources, knowing that getting this right will reap aerodynamic rewards for other areas of the car. The sidepod design draws inspiration from the new bodywork that the team installed in Mugello last season (below) but falls short of having the full ramp to floor transition, instead favouring the dipped midriff like we’ve seen adopted elsewhere. The rear portion of the sidepods and the engine cover have extremely tight packaging, with the AMR21 akin to the W12 with the bodywork almost shrink wrapped to the componentry inside. And, much like the W12, it also features a bodywork blister around the inlet plenum, a feature of the power unit which is believed to be bigger this season as a result of some of the performance and durability updates introduced by HPP. The AMR12 also features a very small rear cooling outlet that not only shows how efficient they expect the Mercedes-AMG F1 M12 E Performance power unit to be, but also how much they have focused on producing a car that recovers the downforce lost by the introduction of the new regulations. The extremely tight packaging creates a sizable undercut beneath the cooling outlet too, which buys back some of the floor that has been lost to the new regulations and drives home the performance of the coke bottle region. This is aided further by the token-free adoption of the Mercedes gearbox carrier and rear suspension from last season, an arrangement that Mercedes was particularly proud of because of the aerodynamic gains that it facilitates. The new arrangement sees the suspension elements lifted clear of the diffuser ceiling, which has become more prominent as the teams push the boundaries of the regulations, while the rear leg of the lower wishbone being positioned so far back also results in the ability to extract more performance from the diffuser. Aston Martin is the first team to unmask all the aerodynamic tricks it will use to make up the difference on the edge of the diagonal floor cut-out. The first of these tricks shares a similarity to the design shown by AlphaTauri, with a trio of outwardly directed fins installed just behind the point where the floor starts to taper in. The airflow structures emitted from these fins will undoubtedly interact with the AlphaTauri-esque floor scroll and floor notch just ahead of them and help to mitigate some of the losses that have been created due to fully enclosed holes being outlawed and the reduced floor width ahead of the rear tyre. It’s here where we find a solution akin to the one that Ferrari tested at the end of 2020 too, as a series of fins form an arc. This should help influence the airflow ahead of the rear tyre and reduce the impact that tyre squirt has on the diffuser. Interestingly, it has also added two offset floor strakes inboard of this where teams normally only opt for one strake, with Mercedes in the pre-hybrid era being an advocate of such designs. A new solution appears on the rear wing too, as the thickness of the upper front corner of the endplate has been altered to allow for another upwash strike. Teams had already started to look for ways to redesign this region last year, with the removal of the louvres in 2019 resulting in an increase in drag. The upwash strike is positioned in order that it can affect the tip vortex that’s generated by the top flap and endplate juncture and will undoubtedly be a design aspect that the rest of the field will take note of. While Aston Martin did show us a lot of its new car, it did keep one element secret for now – the rear brake ducts (not pictured, above). It does seem like a strange omission given it has shown us so much around the rest of the car but we must remember that this is one aspect of the 2021 cars that’s affected by the new regulations. Perhaps the team feels it has found a small pocket of performance in that regard and doesn’t want to unnecessarily hand its rivals a chance to see it ahead of testing.
'''

## Convert the article to list of sentences

In [3]:
sentences=nltk.sent_tokenize(article)

sentences = [sentence.strip() for sentence in sentences]

In [4]:
sentences

["This week's launch of Aston Martin’s new Formula 1 car was one of the most hyped events of the pre-season so far, as fans were intrigued by how the new-look AMR21 would be painted.",
 'Unlike the car launches that came before it, Aston Martin left very little to the imagination, releasing detailed shots of the entire car.',
 'The first thing to note is that the team spent both of its development tokens on redesigning the chassis, in order that it could unlock aerodynamic performance from the central portion of the car.',
 'This is, in part, a legacy of the team’s approach for 2020, having assimilated the overall design package of the previous year’s championship winning Mercedes including a more conventional position for the side-impact protection spars (SIPS).',
 'The low-slung arrangement, as introduced by Ferrari in 2017, is now considered critical from an aerodynamic perspective, with the sidepod inlet positioned much like a periscope.',
 'This is typically above the fairing that

## Using Pandas to efficiently apply various transformations

In [5]:
data = pd.DataFrame(sentences)

data.columns=['sentence']

In [6]:
data

,sentence
0,This week's launch of Aston Martin’s new Formu...
1,"Unlike the car launches that came before it, A..."
2,The first thing to note is that the team spent...
3,"This is, in part, a legacy of the team’s appro..."
4,"The low-slung arrangement, as introduced by Fe..."
5,This is typically above the fairing that surro...
6,This image of the car depicts how the bargeboa...
7,"Meanwhile, the airflow fed from the front of t..."
8,This should result in a much cleaner flow arri...
9,The inlet itself is extremely narrow with the ...


## Convert sentences to contextual dense vectors

In [8]:
def get_sentence_embeddings(sentence):
    embedding = model.encode([sentence])
    return embedding[0]

data['embeddings'] = data['sentence'].apply(get_sentence_embeddings)

In [9]:
data

,sentence,embeddings
0,This week's launch of Aston Martin’s new Formu...,"[-0.11816078, -1.1421314, -0.061358135, 1.0909..."
1,"Unlike the car launches that came before it, A...","[-0.18980047, -0.66455394, -0.11699995, 0.6180..."
2,The first thing to note is that the team spent...,"[0.34146076, -0.50965875, -0.12219028, 0.15770..."
3,"This is, in part, a legacy of the team’s appro...","[0.10547462, -0.8769582, -0.912105, 2.2461271,..."
4,"The low-slung arrangement, as introduced by Fe...","[0.43464914, -0.39981413, -0.61109465, 0.73632..."
5,This is typically above the fairing that surro...,"[0.2963266, 0.14062917, -0.13123445, -0.634986..."
6,This image of the car depicts how the bargeboa...,"[0.9925204, 0.3615471, -0.29278693, 1.4521711,..."
7,"Meanwhile, the airflow fed from the front of t...","[0.64373314, -0.2578171, -0.26096627, -0.33562..."
8,This should result in a much cleaner flow arri...,"[0.29890952, 0.23680705, 0.019264746, -0.31543..."
9,The inlet itself is extremely narrow with the ...,"[0.3280214, -0.088568434, -0.1816906, 0.569646..."


In [10]:
len(data)

31

## Apply NLTK K-mean clustering on embeddings

In [11]:
NUM_CLUSTERS=10

X = np.array(data['embeddings'].tolist())

#initial_centroids = X[:10].tolist()

kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

In [12]:
data['cluster'] = pd.Series(assigned_clusters, index=data.index)
data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

In [13]:
data

,sentence,embeddings,cluster,centroid
0,This week's launch of Aston Martin’s new Formu...,"[-0.11816078, -1.1421314, -0.061358135, 1.0909...",1,"[-0.023809588, -0.9515791, -0.08204963, 0.9149..."
1,"Unlike the car launches that came before it, A...","[-0.18980047, -0.66455394, -0.11699995, 0.6180...",1,"[-0.023809588, -0.9515791, -0.08204963, 0.9149..."
2,The first thing to note is that the team spent...,"[0.34146076, -0.50965875, -0.12219028, 0.15770...",4,"[0.20297739, -0.13747057, -0.23274632, 0.29673..."
3,"This is, in part, a legacy of the team’s appro...","[0.10547462, -0.8769582, -0.912105, 2.2461271,...",6,"[0.7060378, -0.4421278, -0.7036155, 1.8784851,..."
4,"The low-slung arrangement, as introduced by Fe...","[0.43464914, -0.39981413, -0.61109465, 0.73632...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724..."
5,This is typically above the fairing that surro...,"[0.2963266, 0.14062917, -0.13123445, -0.634986...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724..."
6,This image of the car depicts how the bargeboa...,"[0.9925204, 0.3615471, -0.29278693, 1.4521711,...",9,"[0.44443837, 0.18677473, -0.28736037, 0.805646..."
7,"Meanwhile, the airflow fed from the front of t...","[0.64373314, -0.2578171, -0.26096627, -0.33562...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724..."
8,This should result in a much cleaner flow arri...,"[0.29890952, 0.23680705, 0.019264746, -0.31543...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724..."
9,The inlet itself is extremely narrow with the ...,"[0.3280214, -0.088568434, -0.1816906, 0.569646...",0,"[0.1579768, -0.22432399, -0.23417294, 0.265469..."


## Compute distance from the centroid

In [14]:
from scipy.spatial import distance_matrix
def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['embeddings']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)

In [15]:
data

,sentence,embeddings,cluster,centroid,distance_from_centroid
0,This week's launch of Aston Martin’s new Formu...,"[-0.11816078, -1.1421314, -0.061358135, 1.0909...",1,"[-0.023809588, -0.9515791, -0.08204963, 0.9149...",12.045614
1,"Unlike the car launches that came before it, A...","[-0.18980047, -0.66455394, -0.11699995, 0.6180...",1,"[-0.023809588, -0.9515791, -0.08204963, 0.9149...",9.632929
2,The first thing to note is that the team spent...,"[0.34146076, -0.50965875, -0.12219028, 0.15770...",4,"[0.20297739, -0.13747057, -0.23274632, 0.29673...",10.349550
3,"This is, in part, a legacy of the team’s appro...","[0.10547462, -0.8769582, -0.912105, 2.2461271,...",6,"[0.7060378, -0.4421278, -0.7036155, 1.8784851,...",8.482136
4,"The low-slung arrangement, as introduced by Fe...","[0.43464914, -0.39981413, -0.61109465, 0.73632...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724...",12.297242
5,This is typically above the fairing that surro...,"[0.2963266, 0.14062917, -0.13123445, -0.634986...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724...",7.876090
6,This image of the car depicts how the bargeboa...,"[0.9925204, 0.3615471, -0.29278693, 1.4521711,...",9,"[0.44443837, 0.18677473, -0.28736037, 0.805646...",9.858581
7,"Meanwhile, the airflow fed from the front of t...","[0.64373314, -0.2578171, -0.26096627, -0.33562...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724...",9.850224
8,This should result in a much cleaner flow arri...,"[0.29890952, 0.23680705, 0.019264746, -0.31543...",5,"[0.35189387, -0.098310046, -0.2642593, 0.04724...",13.105395
9,The inlet itself is extremely narrow with the ...,"[0.3280214, -0.088568434, -0.1816906, 0.569646...",0,"[0.1579768, -0.22432399, -0.23417294, 0.265469...",11.022434


## Generate Summary

In [18]:
summary=' '.join(data.sort_values('distance_from_centroid',ascending = True).groupby('cluster').head(1).sort_index()['sentence'].tolist())

In [19]:
summary

'The first thing to note is that the team spent both of its development tokens on redesigning the chassis, in order that it could unlock aerodynamic performance from the central portion of the car. This is, in part, a legacy of the team’s approach for 2020, having assimilated the overall design package of the previous year’s championship winning Mercedes including a more conventional position for the side-impact protection spars (SIPS). This is typically above the fairing that surrounds the SIPS, which is used to inhibit the turbulence created by the front tyre and therefore also aids the transit of cool air that’s supplied to the radiators within the sidepods. The inlet itself is extremely narrow with the team recovering some of that with the sculpting on the sides of the chassis. The sidepod design draws inspiration from the new bodywork that the team installed in Mugello last season (below) but falls short of having the full ramp to floor transition, instead favouring the dipped mid